In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
from strategy_v2.Strategy import *
from strategy_v2.TradingSubSystem import *
from strategy_v2.Portfolio import *
from strategy_v2.TransactionCost import *
from strategy_v2.Executor import *
from utils.data_helper import *
from utils.data import *
from utils.performance import *
import cvxpy as cp
from plotly.subplots import make_subplots
from croniter import croniter

# Portfolio General Settings
We assume the code is run at T before market open, so it all data on and before T-1 close. Therefore, the end_date should be today - BDay(1)

In [2]:
end_date = get_today()
start_date = pd.to_datetime(datetime(2022, 1, 3))
vol_target = 0.5
max_leverage = 1

# Linear Optimization (Risk-Adjusted Return)

- How to determine the capital?

Difficulty here is portfolio might use different leverage per rebalance, looking at the asset MV might not give you the total portfolio capital (e.g. 70% leverage, your stocks only worth 70% of your capital now)

we can use the market value on Futu divides by the leverage used for last periods. This will give you the total captial for the portfolio

Then we use this capital for next rebalance

We shuold strategically fix this issue by reading capital from Futu or keep track the capital ourselves

- 2024-03-23 - Leverage Ratio = 102.4376
- 2024-04-20 - Leverage Ratio = 102.6119
- 2024-05-03 - Leverage Ratio = 97.8875
- 2024-05-17 - Leverage Ratio = 99.1256

In [3]:
169971.72/97.8875*100

173639.86208657897

In [4]:
portfolio = PortfolioLinearOpt(
    capital=173639.86208657897,
    name='ETF Buy and Hold Portfolio',
    lookback_period=60,
    opt_freq=1,
    rebalance_iter=RebalancerIter('0 0 * * Fri', 2),    
    tc_model=TransactionCostFutu(),    
    systems=[
        TradingSubSystemSingle(vol_target=vol_target, instruments=['META'],  strategy=[BuyAndHoldStrategy(confidence=1)], max_leverage=max_leverage),
        TradingSubSystemSingle(vol_target=vol_target, instruments=['TSLA'],  strategy=[BuyAndHoldStrategy(confidence=1)], max_leverage=max_leverage),
        TradingSubSystemSingle(vol_target=vol_target, instruments=['NVDA'],  strategy=[BuyAndHoldStrategy(confidence=1)], max_leverage=max_leverage),
        TradingSubSystemSingle(vol_target=vol_target, instruments=['AAPL'],  strategy=[BuyAndHoldStrategy(confidence=1)], max_leverage=max_leverage),
        TradingSubSystemSingle(vol_target=vol_target, instruments=['EWY'],   strategy=[BuyAndHoldStrategy(confidence=1)], max_leverage=max_leverage),
        TradingSubSystemSingle(vol_target=vol_target, instruments=['DXJ'],   strategy=[BuyAndHoldStrategy(confidence=2)], max_leverage=max_leverage),        
        #TradingSubSystemSingle(vol_target=vol_target, instruments=['VOE'],   strategy=[BuyAndHoldStrategy(confidence=1)], max_leverage=max_leverage),
        #TradingSubSystemSingle(vol_target=vol_target, instruments=['VBR'],   strategy=[BuyAndHoldStrategy(confidence=1)], max_leverage=max_leverage),
        TradingSubSystemSingle(vol_target=vol_target, instruments=['BRK-B'], strategy=[BuyAndHoldStrategy(confidence=2)], max_leverage=max_leverage),
        TradingSubSystemSingle(vol_target=vol_target, instruments=['SPY'],   strategy=[BuyAndHoldStrategy(confidence=1)], max_leverage=max_leverage),
        TradingSubSystemSingle(vol_target=vol_target, instruments=['QQQ'],   strategy=[BuyAndHoldStrategy(confidence=1)], max_leverage=max_leverage),        
])

portfolio.set_start_date(start_date)
portfolio.set_end_date(end_date)
portfolio.backtest_subsystems()
portfolio.optimize(gamma=10, hhi=0.2)
portfolio.backtest()
portfolio.rebalance()
portfolio.performance(show_all_rets=True)

[*********************100%***********************]  1 of 1 completed

2024-05-18 01:14:36,946 - TradingSubSystemSingle [META] - INFO - Generating position for strategy BAH1 between 2021-10-04 and 2024-05-17......
2024-05-18 01:14:36,957 - TradingSubSystemSingle [META] - INFO - Volatility Target = 50.0% | Price Volatility = 45.9% | Last Scale Factor = 1.04



[*********************100%***********************]  1 of 1 completed

2024-05-18 01:14:37,229 - TradingSubSystemSingle [TSLA] - INFO - Generating position for strategy BAH1 between 2021-10-04 and 2024-05-17......
2024-05-18 01:14:37,237 - TradingSubSystemSingle [TSLA] - INFO - Volatility Target = 50.0% | Price Volatility = 79.9% | Last Scale Factor = 0.62



[*********************100%***********************]  1 of 1 completed

...


[*********************100%***********************]  1 of 1 completed

2024-05-18 01:14:39,174 - TradingSubSystemSingle [QQQ] - INFO - Generating position for strategy BAH1 between 2021-10-04 and 2024-05-17......
2024-05-18 01:14:39,185 - TradingSubSystemSingle [QQQ] - INFO - Volatility Target = 50.0% | Price Volatility = 14.8% | Last Scale Factor = 2.91


100%|██████████| 620/620 [00:03<00:00, 176.04it/s]


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


,Rebalanced Portfolio,Optimized Portfolio,^SPX,Single - META (BAH1),Single - TSLA (BAH1),Single - NVDA (BAH1),Single - AAPL (BAH1),Single - EWY (BAH1),Single - DXJ (BAH2),Single - BRK-B (BAH2),Single - SPY (BAH1),Single - QQQ (BAH1)
Measure,,,,,,,,,,,,
Cumulative Return,1.658065,1.641389,1.104531,1.487099,0.536792,3.181323,1.083988,0.873027,1.866792,1.389833,1.152228,1.152979
Annualized Return,0.236699,0.232253,0.058933,0.291603,-0.137359,0.605846,0.073605,-0.027436,0.276912,0.154660,0.076843,0.090784
Annualized Volatility,0.216063,0.215181,0.184117,0.492715,0.499627,0.488605,0.281592,0.244582,0.163047,0.177177,0.184452,0.247668
Annualized Sharpe Ratio,0.932300,0.915456,0.128553,0.520259,-0.345503,1.167778,0.136159,-0.256357,1.482079,0.673880,0.225418,0.224171
Maximum Drawdown,-0.261896,-0.271647,-0.254251,-0.682479,-0.635372,-0.531787,-0.309294,-0.406293,-0.117846,-0.265790,-0.244964,-0.348280


In [5]:
portfolio.get_position_for_trade()

2024-05-18 01:14:49,928 - Linear Optimized Portfolio (ETF Buy and Hold Portfolio) - CRITICAL - Portfolio target capital is not specified, use initial backtest capital of $173,640
2024-05-18 01:14:49,928 - Linear Optimized Portfolio (ETF Buy and Hold Portfolio) - INFO - Generate trade position based on target capital of $173,640


,META,TSLA,NVDA,AAPL,EWY,DXJ,BRK-B,SPY,QQQ
Date,,,,,,,,,
2022-01-03,0.0,47.0,85.0,285.0,12.0,81.0,123.0,44.0,12.0
2022-01-04,0.0,47.0,85.0,285.0,12.0,81.0,123.0,44.0,12.0
2022-01-05,0.0,47.0,85.0,285.0,12.0,81.0,123.0,44.0,12.0
2022-01-06,0.0,47.0,85.0,285.0,12.0,81.0,123.0,44.0,12.0
2022-01-07,0.0,47.0,85.0,285.0,12.0,81.0,123.0,44.0,12.0
...,...,...,...,...,...,...,...,...,...
2024-05-13,0.0,19.0,19.0,0.0,346.0,514.0,88.0,50.0,19.0
2024-05-14,0.0,19.0,19.0,0.0,346.0,514.0,88.0,50.0,19.0
2024-05-15,0.0,19.0,19.0,0.0,346.0,514.0,88.0,50.0,19.0


# Equal Weighted Portfolios

In [6]:
# portfolio = PortfolioStandard(    
#     capital=200000,
#     name='ETF Buy and Hold Portfolio',    
#     rebalance_iter=RebalancerIter('0 0 * * Fri', 2),
#     tc_model=TransactionCostFutu(),    
#     systems=[
#         # TradingSubSystemSingle(vol_target=vol_target, instruments=['META'],  strategy=[BuyAndHoldStrategy(confidence=1)], max_leverage=max_leverage),
#         # TradingSubSystemSingle(vol_target=vol_target, instruments=['TSLA'],  strategy=[BuyAndHoldStrategy(confidence=1)], max_leverage=max_leverage),
#         # TradingSubSystemSingle(vol_target=vol_target, instruments=['NVDA'],  strategy=[BuyAndHoldStrategy(confidence=1)], max_leverage=max_leverage),
#         # TradingSubSystemSingle(vol_target=vol_target, instruments=['AAPL'],  strategy=[BuyAndHoldStrategy(confidence=1)], max_leverage=max_leverage),
#         TradingSubSystemSingle(vol_target=vol_target, instruments=['EWY'],   strategy=[BuyAndHoldStrategy(confidence=1)], max_leverage=max_leverage),
#         TradingSubSystemSingle(vol_target=vol_target, instruments=['DXJ'],   strategy=[BuyAndHoldStrategy(confidence=2)], max_leverage=max_leverage),        
#         TradingSubSystemSingle(vol_target=vol_target, instruments=['VOE'],   strategy=[BuyAndHoldStrategy(confidence=1)], max_leverage=max_leverage),
#         TradingSubSystemSingle(vol_target=vol_target, instruments=['VBR'],   strategy=[BuyAndHoldStrategy(confidence=1)], max_leverage=max_leverage),
#         TradingSubSystemSingle(vol_target=vol_target, instruments=['BRK-B'], strategy=[BuyAndHoldStrategy(confidence=2)], max_leverage=max_leverage),
#         TradingSubSystemSingle(vol_target=vol_target, instruments=['SPY'],   strategy=[BuyAndHoldStrategy(confidence=1)], max_leverage=max_leverage),
#         TradingSubSystemSingle(vol_target=vol_target, instruments=['QQQ'],   strategy=[BuyAndHoldStrategy(confidence=1)], max_leverage=max_leverage),       
# ])

# portfolio.set_start_date(start_date)
# portfolio.set_end_date(end_date)
# portfolio.backtest_subsystems()
# portfolio.optimize()
# portfolio.backtest()
# portfolio.rebalance()
# portfolio.performance()

# Execute the Portfolio

In [8]:
executor = ExecutorFutu(is_test=False)
executor.set_portfolio(portfolio)
orders = executor.execute(px_interval='5m')
orders

2024-05-18 01:16:12,994 - ExecutorFutu - INFO - market: US
2024-05-18 01:16:12,995 - ExecutorFutu - INFO - Cancel all orders first before executing.....


2024-05-18 01:16:13,000 | 76619 | [open_context_base.py] _send_init_connect_sync:311: InitConnect ok: conn_id=7, host=127.0.0.1, port=11111, user_id=18214795
2024-05-18 01:16:13,233 | 76619 | [open_context_base.py] on_disconnect:383: Disconnected: conn_id=7


2024-05-18 01:16:16,234 - Linear Optimized Portfolio (ETF Buy and Hold Portfolio) - CRITICAL - Portfolio target capital is not specified, use initial backtest capital of $173,640
2024-05-18 01:16:16,238 - Linear Optimized Portfolio (ETF Buy and Hold Portfolio) - INFO - Generate trade position based on target capital of $173,640
2024-05-18 01:16:16,255 - ExecutorFutu - INFO - Execute Linear Optimized Portfolio (ETF Buy and Hold Portfolio) position based on 2024-05-17


2024-05-18 01:16:16,262 | 76619 | [open_context_base.py] _send_init_connect_sync:311: InitConnect ok: conn_id=8, host=127.0.0.1, port=11111, user_id=18214795


2024-05-18 01:16:16,274 - Futu - INFO - 7 Positions: US.TSLA, US.SPY, US.QQQ, US.NVDA, US.EWY, US.DXJ, US.BRK.B


...

2024-05-18 01:16:33,349 - Futu - INFO - Placed Order: {'code': 'US.SPY', 'price': 528.77, 'qty': 27.0, 'trd_side': 'BUY', 'order_type': 'NORMAL', 'market': 'US', 'trd_env': 'REAL'}


2024-05-18 01:16:33,357 | 76619 | [open_context_base.py] on_disconnect:383: Disconnected: conn_id=14
2024-05-18 01:16:36,366 | 76619 | [open_context_base.py] _send_init_connect_sync:311: InitConnect ok: conn_id=15, host=127.0.0.1, port=11111, user_id=18214795


2024-05-18 01:16:36,627 - Futu - INFO - Placed Order: {'code': 'US.QQQ', 'price': 451.69, 'qty': 4.0, 'trd_side': 'SELL', 'order_type': 'NORMAL', 'market': 'US', 'trd_env': 'REAL'}


2024-05-18 01:16:36,630 | 76619 | [open_context_base.py] on_disconnect:383: Disconnected: conn_id=15


,code,stock_name,trd_side,order_type,order_status,order_id,qty,price,create_time,updated_time,...,remark,time_in_force,fill_outside_rth,aux_price,trail_type,trail_value,trail_spread,currency,portfolio,date
0,US.TSLA,特斯拉,SELL,NORMAL,SUBMITTING,8616446230932792145,19.0,177.54,2024-05-17 13:16:17,2024-05-17 13:16:17,...,Linear Optimized Portfolio (ETF Buy and Hold P...,DAY,False,N/A,N/A,N/A,N/A,USD,Linear Optimized Portfolio (ETF Buy and Hold P...,2024-05-17
0,US.AAPL,苹果,BUY,NORMAL,SUBMITTING,7250476053938312820,96.0,189.85,2024-05-17 13:16:20,2024-05-17 13:16:20,...,Linear Optimized Portfolio (ETF Buy and Hold P...,DAY,False,N/A,N/A,N/A,N/A,USD,Linear Optimized Portfolio (ETF Buy and Hold P...,2024-05-17
0,US.EWY,韩国ETF-iShares MSCI,SELL,NORMAL,SUBMITTING,866087946546130038,308.0,65.47,2024-05-17 13:16:23,2024-05-17 13:16:23,...,Linear Optimized Portfolio (ETF Buy and Hold P...,DAY,False,N/A,N/A,N/A,N/A,USD,Linear Optimized Portfolio (ETF Buy and Hold P...,2024-05-17
0,US.DXJ,日本对冲股票ETF-WisdomTree,SELL,NORMAL,SUBMITTING,5104385882552043929,60.0,108.85,2024-05-17 13:16:26,2024-05-17 13:16:26,...,Linear Optimized Portfolio (ETF Buy and Hold P...,DAY,False,N/A,N/A,N/A,N/A,USD,Linear Optimized Portfolio (ETF Buy and Hold P...,2024-05-17
0,US.BRK.B,伯克希尔-B,SELL,NORMAL,SUBMITTING,319846399867371719,14.0,415.59,2024-05-17 13:16:30,2024-05-17 13:16:30,...,Linear Optimized Portfolio (ETF Buy and Hold P...,DAY,False,N/A,N/A,N/A,N/A,USD,Linear Optimized Portfolio (ETF Buy and Hold P...,2024-05-17
0,US.SPY,SPDR 标普500指数ETF,BUY,NORMAL,SUBMITTING,1339895322401886599,27.0,528.77,2024-05-17 13:16:33,2024-05-17 13:16:33,...,Linear Optimized Portfolio (ETF Buy and Hold P...,DAY,False,N/A,N/A,N/A,N/A,USD,Linear Optimized Portfolio (ETF Buy and Hold P...,2024-05-17
0,US.QQQ,纳指100ETF-Invesco QQQ Trust,SELL,NORMAL,SUBMITTING,207605671369799532,4.0,451.69,2024-05-17 13:16:36,2024-05-17 13:16:36,...,Linear Optimized Portfolio (ETF Buy and Hold P...,DAY,False,N/A,N/A,N/A,N/A,USD,Linear Optimized Portfolio (ETF Buy and Hold P...,2024-05-17
